In [494]:
import numpy as np

In [495]:
# Compute the gradient using central difference formula
def gradient(f, x, h=1e-5):
    n = len(x)
    grad = np.zeros(n)
    for i in range(n):
        x_forward = np.copy(x)
        x_backward = np.copy(x)
        x_forward[i] += h
        x_backward[i] -= h
        grad[i] = (f(x_forward) - f(x_backward)) / (2 * h)
    return grad

In [496]:
def hessian(f, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                # Diagonal elements: Second partial derivatives w.r.t. the same variable
                x_ij_forward = np.copy(x)
                x_ij_backward = np.copy(x)
                x_ij_forward[i] += h
                x_ij_backward[i] -= h
                f_ij_forward = f(x_ij_forward)
                f_ij_backward = f(x_ij_backward)
                H[i, j] = (f_ij_forward - 2*f(x) + f_ij_backward) / (h**2)

            else:
                # Off-diagonal elements: Mixed partial derivatives
               x_ij_forward = np.copy(x)
               x_ij_backward = np.copy(x)
               x_ij_forward[i] += h
               x_ij_forward[j] += h
               x_ij_backward[i] -= h
               x_ij_backward[j] -= h
               f_ij_forward = f(x_ij_forward)
               f_ij_backward = f(x_ij_backward)

               H[i, j] = (f_ij_forward - f(x_ij_forward - 2*h*np.eye(n)[i]) -
                           f(x_ij_forward - 2*h*np.eye(n)[j]) + f_ij_backward) / (4 * h**2)
    return H

In [497]:
#quadratic approximation of model
def mk(x,f,p):
  grad = gradient(f,x)
  hess = hessian(f,x)
  return f(x) + np.dot(grad,p) + 0.5*np.dot(p, np.dot(hess, p))

In [498]:
#steepest direction
def steep(f,x):
  grad = gradient(f,x)
  hess = hessian(f,x)
  num = np.dot(grad, grad)
  denom = np.dot(grad, np.dot(hess, grad))

  if denom == 0:
    alpha = 1
  else:
    alpha = num/denom

  return -alpha*grad

In [499]:
#newton direction
def newton(f,x):
  grad = gradient(f,x)
  hess = hessian(f,x)
  hess_inv = np.linalg.inv(hess)
  direction = -np.dot(hess_inv,grad)
  return direction

In [500]:
#function to find theta
def theta(steep,newton,delta):
  A = np.dot(newton,newton)
  B = np.dot(newton,steep)
  C = np.dot(steep,steep)

  a = A - 2*B + C
  b = 2*B - 2*C
  c = C - delta**2

  if b**2 - 4*a*c < 0:
    raise ValueError("No real solutions, check inputs")

  num = -b + np.sqrt(b**2 - 4*a*c)
  denom = 2*a

  return num/denom

In [501]:
def dogleg(f, x0, delta = 1, delta_bar = 5,max_iter = 1000, tol = 1e-4):

  x = np.copy(x0)
  n = len(x)

  for _ in range(max_iter):

    g = gradient(f, x)

    if np.linalg.norm(g) < tol: return x

    p_c = steep(f,x)
    p_n = newton(f,x)

    if np.linalg.norm(p_c) <= delta:
      p = p_n
    elif np.linalg.norm(p_c) >= delta:
      p = delta*(p_c/np.linalg.norm(p_c))
    else:
      thetaa = theta(p_c,p_n,delta)
      p = thetaa*p_n + (1-thetaa)*p_c

    x_new = x + p

    rho = (f(x) - f(x_new))/(mk(x,f,p*0) - mk(x,f,p))

    if rho > 0.75 and np.linalg.norm(p) == delta:
      delta = np.minimum(2*delta,delta_bar)

    if rho < 0.25:
      delta = 0.25*delta

    if rho > 0:
      x = x_new

In [506]:
#Q1
def f1(x):
  return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 -7)**2

x = dogleg(f1, [5.0, 2.0])

f = f1(x)
print(f'Optimal x is {x} and optimal function value is {f}')

Optimal x is [3.00000026 1.99999993] and optimal function value is 2.2650255576723473e-12


In [507]:
#Q2
def f2(x):
  return 100*(x[1]-x[0]**2)**2 + (1-x[0])**2

x = dogleg(f2, [0.0,3.0])
f = f2(x)

print(f'Optimal x is {x} and optimal function value is {f}')

Optimal x is [0.99999987 0.99999971] and optimal function value is 6.511664525102119e-14


In [504]:
#Q3
def f3(x):
  return 0.1*(12 + x[0]**2 + ((1 + x[1])**2)/(x[0]**2) + (x[0]**2 + x[1]**2 + 100)/((x[0]*x[1])**4))

x = dogleg(f3, [6.0,4.0])
f = f3(x)

print(f'Optimal x is {x} and optimal function value is {f}')

Optimal x is [1.90526302 1.82963007] and optimal function value is 1.8560211477891408


In [505]:
#Q4
def f4(x):
  x1, x2, x3, x4 = x
  return (x1 + 10*x2)**2 + 5*(x3-x4)**2 + (x2-2*x3)**4 + 10*(x1-x4)**4

x = dogleg(f4, [2,2,1,1])
f = f4(x)

print(f'Optimal x is {x} and optimal function value is {f}')

Optimal x is [ 0.01525759 -0.00152576  0.00869266  0.00869266] and optimal function value is 1.464730218158901e-07
